In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///../data/catastici.db")

# test DB
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM catastici LIMIT 1;")

sqlite
['catastici']


"[(0, 3183, 'liberal', 'campi', 'casa e bottega da barbier', 70, 'campo vicino alla chiesa')]"

In [2]:
from collections import Counter
import pandas as pd
import ast
import re
import warnings
warnings.filterwarnings('ignore')

def clean_query(sql_query):
    """clean the output"""
    # change to list
    sql_query_list = ast.literal_eval(sql_query)
    
    # split on ;
    sql_query_list = [query.split(';')[0].strip() + ';' for query in sql_query_list]
    
    # replace '
    sql_query_list = [re.sub(r"([a-z])'([a-z])",r"\1''\2", query) for query in sql_query_list]

    return sql_query_list

def check_sql_executability(query, db):
    try:
        return db.run(query)
    except Exception as e:
        return str(e)
     
def find_most_common_answer(answers):
    answers = ["ERROR" if "error" in answer else answer for answer in answers]
    most_common_answer, most_common_count = Counter(answers).most_common(1)[0]
    if most_common_answer == "ERROR":
        if most_common_count == 4:
            return most_common_answer
        else:
            return Counter(answers).most_common(2)[1][0]

    return most_common_answer

def clean_answer(answer, to_replace = ['[', ']', '(', ',)', "'", ')']):
    pattern = '|'.join(map(re.escape, to_replace))
    cleaned_answer = re.sub(pattern, '', answer)
    return [ans.strip() for ans in cleaned_answer.split(',')]

def ngram_overlap(true_answer, generated_answer):
    """Calculates 1-gram overlap"""
    # Convert lists to sets to remove duplicates
    true_set = set(true_answer)
    generated_set = set(generated_answer)
    
    # Calculate the intersection of the sets
    overlap_count = len(true_set.intersection(generated_set))
    
    # Calculate the percentage of overlap
    overlap_percentage = overlap_count / len(true_set) if len(true_set) > 0 else 0.0
    
    return round(overlap_percentage, 3)

In [4]:
# import the dataset
query_res = pd.read_csv('./test_data_generated.csv')

# clean output
for idx, row in query_res.iterrows():
    query_list_clean = clean_query(row['generated_query'])
    final_out = None
    answers = []
    for out in query_list_clean:
        answers.append(check_sql_executability(out, db))
    answer = find_most_common_answer(answers)
    
    if answer != "ERROR":
        final_out = query_list_clean[answers.index(answer)]
    if final_out == None:
        final_out = query_list_clean[0]
        answer = check_sql_executability(final_out, db)
    query_res.loc[idx,'generated_answer'] = answer
    query_res.loc[idx,'generated_query'] = final_out
    
query_res.loc[(query_res['generated_answer'].str.contains("error")), 'output'] = 'ERROR'
query_res.loc[(query_res['generated_answer']==query_res['true_answer']), 'output'] = 'EM'
query_res['output'].value_counts()

output
EM       236
ERROR     18
Name: count, dtype: int64

In [7]:
query_wrong = query_res[query_res.output.isna()]
query_wrong.true_answer.fillna("",inplace=True)

query_wrong['generated_answer_clean'] = query_wrong['generated_answer'].apply(clean_answer)
query_wrong['true_answer_clean'] = query_wrong['true_answer'].apply(clean_answer)

n_gram = []
for _, row in query_wrong.iterrows():
    n_gram.append(ngram_overlap(row['true_answer_clean'], row['generated_answer_clean']))
query_wrong['n_gram_overlap'] = n_gram
query_wrong['n_gram_matching'] = query_wrong['n_gram_overlap']>0.33

query_wrong['n_gram_matching'].value_counts()

n_gram_matching
True     123
False    123
Name: count, dtype: int64

# Check

- Wrong -> 246<br>
    - True -> 162
    - Wrong -> 84 <= 12 + [123, 166, 167, 169, 175-179, 186, 209, 213, 225, 227, 256-258, 266, 276-278, 281 (uppercased), 308, 330, 338, 345-346, 348-349, 351, 370-371, 375-377, 383, 387, 388, 408, 415-419, 422-423, 441, 455-459, 461-464, 476, 479-489, 495-499]
- Error -> 18<br>
- Exact Match -> 230
- True -> **392**

### Wrong Groud Truth
17(distinct), 20, 40, 75, 95, 121, 135, 170(answer is nan), 185(answer is nan), 240, 306, 311 (do sum), 390-393 (median), 417 (this specific question), 480 (tricky median)

### Ambigious questions <br>
200 - 204, 85 - 89 (total property or single property?), 90-94, 375, 430, 433

### Both correct
- Question: What does the typical rental revenue look like for properties situated at "al ponte di san provolo"? -> TRUE (average), GEN (average by property type)
- What is the range of rent incomes in "calle de franchi"? -> TRUE (min, max), GEN (max-min)

### Wrong generation

Sometimes, it puts extra filter (on Limit to avoid long answer, but it is not asked) - 3, 8, 11, 12, 13, 14, 25, 35<br>
- It, sometimes, uses limit 1 when question is singular
Sometimes, it confuses the feature names: i.e Rent_Income instead of Property_Type <br>
Example questions:
- Which properties have a rent income higher than 50 ducati? (limit 1)
- Question: What are the different classifications of property present in "teren alli gesouiti"? (uses "Property_Classification")
- How many properties are enumerated in the dataset? (uses "WHERE "Property_Type" = 'enumerazione'")
- Who is the owner earning the highest rental income from a property? (does sum instead of max)
- What is the count of unique locations where properties are situated? (does not count)
- What does the typical rental revenue amount to for each category of the "perina" "capello" properties? (it does "Property_Location" = 'perina')
- What is the total number of properties generating less than 30 ducati in rent? (does sum of rent instead of count)

### Limitations - Error
- Non-existing SQL keywords, such as STDDEV: e.g. For each property location, what's the standard rental income? -> uses STDDEV instead of AVG (question is also ambigious)<br>
Statistics, i.e. median, standard deviation, variance, ... <br>
- More than 1 arguments in Count <br>
- Non-existing feature names, when question is ambigious: e.g. How many real estate properties are on lease for over "38"? <br>
- Example questions: 
    - Could you provide the median leasing earnings across all assets in "al ponte di san provolo"? (uses PERCENTILE_CONT)
    - Can you tell me the total number of different property owners present in the dataset? (>2 args in Count)
    - For each property location, what's the standard rental income? (uses SDDEV)
    - How many real estate properties are on lease for over "38"? (uses non-existing feature)
    - Whose real estate assets are spread over the widest array of locations?
    - What is the number of households possessing assets across multiple type categories?
    - What is the average rent income variance across all locations? (wrong statistics)

In [32]:
for idx, row in query_res[(query_res['generated_answer']=='ERROR')].iterrows():
    print(f"{row['level']} - {row['question_id']} - {idx}")
    print(f"Question: {row['question']}")
    print(f"Answer True: {row['true_answer']}")
    print(f"Answer Generated: {row['generated_answer']}")
    print('True SQL:')
    print(row['true_query'])
    print('Generated SQL:')
    print(row['generated_query'])
    print('\n\n')    

0 - 25 - 121
Question: Could you provide the median leasing earnings across all assets in "al ponte di san provolo"?
Answer True: [(103.33333333333333,)]
Answer Generated: ERROR
True SQL:
SELECT AVG("Rent_Income") 
FROM catastici 
WHERE "Property_Location" = 'al ponte di san provolo';
Generated SQL:
SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY "Rent_Income") AS Median_Leasing_Earnings
FROM catastici
WHERE "Property_Location" = 'al ponte di san provolo';
SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY "Rent_Income") AS Median_Leasing_Earnings
FROM catastici
WHERE "Property_Location" = 'al ponte di san provolo';
SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY "Rent_Income") AS Median_Leasing_Earnings
FROM catastici
WHERE "Property_Location" = 'al ponte di san provolo';
SELECT PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY "Rent_Income") AS Median_Leasing_Earnings
FROM catastici
WHERE "Property_Location" = 'al ponte di san provolo';



0 - 41 - 200
Question: What is the number of